## 90-s Augmented Dataset

The idea of 90-s overlaying audio is random the object number 3-7 segments, change the `loudness` of object audio(e.g.vehicle) to be equal to the base audio(environment), random the begin time. Then, random apply gain to the object audio before doing the overlay. (Option 2)
<img src="image.png" width="500" height="300">

#### Setup

In [1]:
import os
import random
from pydub import AudioSegment

tgt_path = "I:/datasets/vehicle/dataset/v1/test_set/vehicle"
env_path = "I:/datasets/vehicle/random_environment/jun-jul2020/before_augmented/wav/vehicle"
dst_path = "I:/datasets/vehicle/random_environment/jun-jul2020/augmented/vehicle"


tgt_files = os.listdir(tgt_path)
env_files = os.listdir(env_path)

if not os.path.exists(dst_path):
    os.makedirs(dst_path)
    print("Create:", dst_path)

Create: I:/datasets/vehicle/random_environment/jun-jul2020/augmented/vehicle


#### Overlay

In [2]:
def custom_apply_gain(base_decibel, audio, gain):
    # Set the audio loudness equal to the base_decibel
    decibel_differential = base_decibel - audio.dBFS
    audio = audio.apply_gain(decibel_differential)
    # Apply gain to the audio
    return audio.apply_gain(gain)

In [3]:
# Loop over the environment audio: random audio number, apply gain, and overlay
for i, env_file in enumerate(env_files):
    num_of_audio = random.randint(3,7)
    try:
        tgt_name = random.sample(tgt_files, num_of_audio)
        tgt_audio = AudioSegment.from_file(tgt_path + '/' + tgt_name[0])
        env_audio = AudioSegment.from_file(env_path + '/' + env_file)
        
        # Gain base is from 20% decibel of environment. Gain value need to be integer.
        gain_base = int(abs(env_audio.dBFS * 0.2))

        gain = random.randint(0, gain_base)
        
        # Set the first audio gain
        tgt_audio = custom_apply_gain(env_audio.dBFS, tgt_audio, gain)
        
        for audio in tgt_name[1:]:
            segment = AudioSegment.from_file(tgt_path + '/' + audio)
            
            gained_audio = custom_apply_gain(env_audio.dBFS, segment, gain)
            
            tgt_audio = tgt_audio + gained_audio

        # position should beware of too late start overlay
        position = random.randint(0, int(env_audio.duration_seconds - (num_of_audio * 2))) * 1000 
        overlay_audio = env_audio.overlay(tgt_audio, position=position)
        overlay_audio.export("{}/{}".format(dst_path, env_file),format='wav')
        
        if(i%100 == 0):
            print("Success:", i, "files", num_of_audio, position)
    except Exception as e:
        print(e)
        pass
print("Finish the task")

Success: 0 files 5 64000
Success: 100 files 6 9000
Success: 200 files 5 56000
Success: 300 files 5 8000
Success: 400 files 5 65000
Success: 500 files 3 55000
Success: 600 files 7 30000
Success: 700 files 5 75000
Success: 800 files 6 78000
Success: 900 files 7 25000
Success: 1000 files 3 38000
Success: 1100 files 6 2000
Success: 1200 files 7 23000
Success: 1300 files 3 60000
Success: 1400 files 5 8000
Success: 1500 files 7 33000
Success: 1600 files 4 65000
Finish the task
